In [1]:
from env import IaaS , Workload
from env.workflow import Workflow
from schedule.rdwsEnv import runEnv
from schedule.schedule import Scheduler
from model.sac import SAC
from model.dqn import DQN
import torch
import torch.nn as nn
import datetime
import time  

# __init__ 初始化
random_seed = 50
buffer_size = 1000
batch_size = 512
hidden_size = 128
target_update = 100
action_num = 6
state_dim =  6+3*action_num 
# 将某个事件的到达率从每分钟 0.1 次转换为每秒的到达率
arrival_rate = 0.1/60
test_path = 'data/test'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
episode_number = 100
wf_number = 10
# 平衡预算和时间的参数
beta = 0.7
# 平衡未来奖励的权重
discount_factor = 0.99
lr = 1e-3
debug = False

In [2]:
agent1 = DQN(state_dim, action_num, batch_size, buffer_size, hidden_size, lr, device, discount_factor);
agent2 = SAC(state_dim, action_num, batch_size, buffer_size, hidden_size, lr, device)
save_model_path1 = 'result/dqn_model.pth'
save_model_path2 = 'result/sac_model.pth'
save_model_path3 = 'result/double dqn_model.pth'

In [3]:
def test(episode_number, agent, workflow_number, test_wf_path, save_model_path, arrival_rate, debug=False):
  mean_makespan = [];
  mean_cost = []
  time_rate = []
  cost_rate = []
  succes_both_rate = []
  episode_arr = []
  
  print("start at:", str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")));
  start = time.time();
  
  agent = agent
  agent.load_model(save_model_path)
  agent.net.train(False)
  scheduler = Scheduler(agent, action_num, state_dim, batch_size, beta)
  
  for episode in range(1, episode_number+1):
    print("episode:", episode, "wf num:", workflow_number, "-------------------------------")
    Workflow.reset();
    IaaS.reset();
    Workload.reset();
    
    t, c, dsr, bsr, both = runEnv(test_wf_path, scheduler.run, 10*episode, wf_number=workflow_number, action_num = action_num,
                                  arrival_rate = arrival_rate, debug=False);
    mean_makespan.append(t)
    mean_cost.append(c)
    time_rate.append(dsr)
    cost_rate.append(bsr)
    succes_both_rate.append(both)
    episode_arr += ([episode])

  print('='*70);
  s = str(datetime.timedelta(seconds=time.time()-start));
  print("total run time:", s);
  scheduler.trainPlotFinal(mean_makespan, mean_cost,time_rate, cost_rate, succes_both_rate);
  return mean_makespan, mean_cost,time_rate, cost_rate, succes_both_rate

In [4]:
t1, c1, tr1, cr1, br1 = test(episode_number, agent1, wf_number, test_path, save_model_path3, arrival_rate, debug)
# t2, c2, tr2, cr2, br2 = test(episode_number, agent2, wf_number, test_path, save_model_path2, arrival_rate, debug)

start at: 2024-09-23 16:54:05
episode: 1 wf num: 10 -------------------------------
[Budget] wf.path: data/test/CYBERSHAKE.n.50.14.dax, wf.budget = 23.92, wf.cost = 57.69, distance = -33.77
[Budget] wf.path: data/test/CYBERSHAKE.n.50.1.dax, wf.budget = 52.66, wf.cost = 60.27, distance = -7.61
[Deadline] wf.path: data/test/CYBERSHAKE.n.50.1.dax, wf.deadline = 4221.60, wf.makespan = 14241.83, distance = -10020.23
[Budget] wf.path: data/test/CYBERSHAKE.n.50.16.dax, wf.budget = 36.18, wf.cost = 40.14, distance = -3.96
[Budget] wf.path: data/test/CYBERSHAKE.n.50.17.dax, wf.budget = 25.53, wf.cost = 50.69, distance = -25.16
[Budget] wf.path: data/test/CYBERSHAKE.n.100.4.dax, wf.budget = 31.89, wf.cost = 77.46, distance = -45.57
[Budget] wf.path: data/test/CYBERSHAKE.n.50.6.dax, wf.budget = 8.64, wf.cost = 21.27, distance = -12.63
cost fail total : 6.0, makespan fail total : 1.0
episode: 2 wf num: 10 -------------------------------
[Budget] wf.path: data/test/CYBERSHAKE.n.200.8.dax, wf.budget

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def test_plot(method, y1, y2):
    x = range(1, episode_number+1)
    plt.figure()
    plt.plot(x, y1, label='y1', marker='o')
    plt.plot(x, y2, label='y2', marker='s')

    # 添加标题和标签
    plt.title('Multiple models values ' + method)
    plt.xlabel(episode_number,'episode_number')
    plt.ylabel('Y values')
    plt.legend()
    plt.savefig('result/picture/' + method + '_plot.png')
    plt.show()

In [ ]:
test_plot('makespan', t1, t2)
test_plot('cost', c1, c2)
test_plot('time_rate', tr1, tr2)
test_plot('cost_rate', cr1, cr2)
test_plot('succes_both_rate', br1, br2)